<a href="https://colab.research.google.com/github/thudoan3o8/Crawl-/blob/main/Springer_Crawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
link_base = "https://link.springer.com/journals/"

In [ ]:
list_category = []
for char in range(97, 123):
    category = link_base + chr(char)
    list_category.append(category)

In [ ]:
list_category

['https://link.springer.com/journals/a',
 'https://link.springer.com/journals/b',
 'https://link.springer.com/journals/c',
 'https://link.springer.com/journals/d',
 'https://link.springer.com/journals/e',
 'https://link.springer.com/journals/f',
 'https://link.springer.com/journals/g',
 'https://link.springer.com/journals/h',
 'https://link.springer.com/journals/i',
 'https://link.springer.com/journals/j',
 'https://link.springer.com/journals/k',
 'https://link.springer.com/journals/l',
 'https://link.springer.com/journals/m',
 'https://link.springer.com/journals/n',
 'https://link.springer.com/journals/o',
 'https://link.springer.com/journals/p',
 'https://link.springer.com/journals/q',
 'https://link.springer.com/journals/r',
 'https://link.springer.com/journals/s',
 'https://link.springer.com/journals/t',
 'https://link.springer.com/journals/u',
 'https://link.springer.com/journals/v',
 'https://link.springer.com/journals/w',
 'https://link.springer.com/journals/x',
 'https://link.s

In [ ]:
#Đếm tổng số lượng Journal
list_journal =[]
for i in range(len(list_category)):
  req = requests.get(list_category[i])
  soup = BeautifulSoup(req.text, 'html.parser')
  #đếm số trang của từng category
  page_num = soup.find('nav', class_ = 'c-pagination-listed')
  if page_num:
    page_num = page_num.find_all('li')
    page_num = len(page_num) - 2

  else:
    page_num = 1
  #trích danh sách tất cả link journal
  for page in range (1,page_num+1):
    req = requests.get(list_category[i]+ '/' + str(page))
    soup = BeautifulSoup(req.text, 'html.parser')
    for links in soup.find_all('a',class_ ='c-atoz-list__link'):
      journal = links.get('href')
      list_journal.append(journal)
print(len(list_journal))

3825


In [ ]:
import requests

from requests.adapters import HTTPAdapter
import requests.packages.urllib3.util.retry as urllib3_retry
import time

retry_strategy = urllib3_retry.Retry(
    total=3,
    backoff_factor=0.5,
    status_forcelist=[500, 502, 503, 504],
)

http = requests.Session()
http.mount("http://", HTTPAdapter(max_retries=retry_strategy))
http.mount("https://", HTTPAdapter(max_retries=retry_strategy))

data = []
STT = 0
for i in range(len(list_journal)):
    retry_count = 0
    while retry_count < 3:
        try:
            req = http.get(list_journal[i], timeout=(5, 15))
            soup = BeautifulSoup(req.text, 'html.parser')

            #LẤY TITLE
            title = soup.find('h1', id='journalTitle')
            if title is None:
              title = soup.find('div', id='journalTitle')
              if title is None:
                title = soup.find('h1')
                title = title.get_text().strip()
              else:
                title = title.get_text().strip()
            else:
              title = title.get_text().strip()


            #LẤY IMPACT FACTOR
            impact_factor = None

            impact_factor = soup.find('dd', attrs={'data-test': 'impact-factor-value'})
            if impact_factor:
              impact_factor_text = impact_factor.get_text()
              impact_factor_value = re.search(r'^([\d.]+)', impact_factor_text)
              if impact_factor_value:
                impact_factor = impact_factor_value.group(1)

            if impact_factor is None:
              impact_factor_strong = soup.find('strong',string = 'Citation Impact')
              if impact_factor_strong:
                next_sibling = impact_factor_strong.find_next_sibling('br')
                if next_sibling:
                  next_sibling_text = str(next_sibling.next_sibling)
                  impact_factor_value = re.search(r'(\d+\.\d+)\s+-', next_sibling_text)
                  if impact_factor_value :
                    impact_factor = impact_factor_value.group(1)

            if impact_factor is None:
              impact_factor = " "


            #LẤY ISSN
            issn = None

            issn_element = soup.find('p', class_='c-journal-footer__issn')
            if issn_element:
              issn_match = re.search(r'ISSN:\s*(\d{4}-\d{4})', issn_element.get_text())
              if issn_match:
                issn = issn_match.group(1)

            if issn is None:
              issn_div = soup.find('div', class_='u-display-flex', attrs={'data-test': 'springer-issn'})
              if issn_div:
                issn = issn_div.find('dt', class_='c-list-description__term', string='Print ISSN')
                if issn:
                  issn = issn.find_next('dd', class_='c-list-description__details')
                  if issn:
                    issn = issn.get_text().strip()

            if issn is None:
              issn = " "

            #LẤY e-ISSN
            e_issn = None

            e_issn_ele =soup.find('dt', class_ ='c-list-description__term', string = 'Electronic ISSN' )
            if e_issn_ele:
              e_issn_dd = e_issn_ele.find_next('dd', class_='c-list-description__details')
              if e_issn_dd:
                e_issn = e_issn_dd.get_text().strip()

            if e_issn is None:
              e_issn_div = soup.find('div', class_='u-display-flex', attrs={'data-test': 'springer-issn'})
              if e_issn_div:
                e_issn = e_issn_div.find('dt', class_='c-list-description__term', string='Electronic ISSN')
                if e_issn:
                  e_issn = e_issn.find_next('dd', class_='c-list-description__details')
                  if e_issn:
                    e_issn = e_issn.get_text().strip()

            if e_issn is None:
                e_issn_li= soup.find('li', class_ = 'c-list-group__item', string = re.compile(r'ISSN:\s+\d{4}-\d{4} \(electronic\)'))
                if e_issn_li:
                  e_issn = re.search(r'ISSN:\s+(\d{4}-\d{4}) \(electronic\)', e_issn_li.get_text())
                  if e_issn:
                    e_issn = e_issn.group(1)



            if e_issn is None:
              e_issn = " "

            break
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            retry_count += 1
            print(f"Retrying... (Attempt {retry_count})")
            time.sleep(5)

    if retry_count == 3:
        print(f"Failed to retrieve data for {list_journal[i]} after {retry_count} attempts.")

    # Thêm vào list
    data.append({
        "STT": STT + 1,
        "Tên tạp chí": title,
        "Impact Factor": impact_factor,
        "ISSN": issn,
        "eISSN": e_issn})

    STT += 1

Request failed: HTTPSConnectionPool(host='www.springer.com', port=443): Read timed out.
Retrying... (Attempt 1)


In [ ]:
#Tạo dataframe chứa thông tin journal
df_journal = pd.DataFrame(data, columns = ['STT','Tên tạp chí', 'ISSN', 'eISSN', 'Impact Factor'])
df_journal = df_journal.set_index('STT')


In [ ]:
df_journal

,Tên tạp chí,ISSN,eISSN,Impact Factor
STT,,,,
1,AAPPS Bulletin,,2309-4710,
2,AAPS Open,,2364-9534,
3,AAPS PharmSciTech,,1530-9932,3.3
4,ADHD Attention Deficit and Hyperactivity Disor...,,,
5,AI & SOCIETY,0951-5666,1435-5655,3.0
...,...,...,...,...
3821,Zoomorphology,0720-213X,1432-234X,1.0
3822,Zorg + Welzijn,1381-4664,2468-1369,
3823,Zorg en Financiering,,,


In [ ]:
df_journal.to_csv('df_journal.csv')